In [ ]:
import pm4py
from src.data_pipeline.s3 import fetch_file

OWNER = "jcostello93"
REPO = "node-red-contrib-node-reddit"
S3_BUCKET = "process-mining-github-issues-staging"

file = f"{OWNER}_{REPO}_event_log.xes"
issues = fetch_file(file, S3_BUCKET, file)

log = pm4py.read_xes(file)
print(log.head())